In [8]:

# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/experiments'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
    
import plotly.express as px
import plotly.graph_objects as go
import re
import time
import shutil
import json
import glob
import pandas as pd
import os
import glob
from pprint import pprint
from lib import eval_helper
from lib import utils
from lib import slurm
from lib.slurm import run_sbatch_job


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on host:  submit-0


# Create eval config

In [10]:
data_files = {}
for f in glob.glob("/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_*.json"):
    conf = utils.read_json(f)
    bm = f.split("/")[-1].replace("eval_", "").replace(".json", "")
    # print(bm)
    data_files[bm] = {
        "train_file": conf["eval_args"]["train_file"],
        "validation_file": conf["eval_args"]["validation_file"]
    }

data_files

{'infographics_w_ocr': {'train_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl',
  'validation_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl'},
 'infographics': {'train_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl',
  'validation_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl'},
 'mmbench': {'train_file': '/fsx_0/dataset01/mmbench/processed_dev_20231212.json',
  'validation_file': '/fsx_0/dataset01/mmbench/processed_dev_20231212.json'},
 'mathvista': {'train_file': '/fsx_0/dataset01/mathvista/test.jsonl',
  'validation_file': '/fsx_0/dataset01/mathvista/test.jsonl'},
 'ai2d': {'train_file': '/fsx_0/dataset01/ai2d/ai2d_test.jsonl',
  'validation_file': '/fsx_0/dataset01/ai2d/ai2d_test.jsonl'},
 'docvqa': {'train_file': '/fsx_0/dataset01/docvqa/val.jsonl',
  'validation_file': '/fsx_0/dataset01/docvqa/val.jsonl'},
 'textvqa': {'train_file': '/fsx_0/dataset01/textvqa/text_vqa_val_set_updated.jsonl',
  'validation_file': '/fsx_0/dat

In [14]:
original_dir = "/fsx_0/user/tranx/experiments/eval/sft_eval_overwrite_fb"
new_dir = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite"

for file in glob.glob(f"{original_dir}/*.json"):
    benchmark = os.path.basename(file).replace(".json", "")
    config = utils.read_json(file)
    
    for k in ["scheduler_type", "num_gpus", "num_nodes"]:
        config.pop(k)
        
    config["eval_args"].update({
        "output_dir": "OUTPUT_DIR",
        "resume_from_checkpoint": "CHECKPOINT_PATH",
        "logging_dir": "LOGGING_DIR",
        "tb_logdir": "TB_LOGDIR",
        "eval_ckpt": "EVAL_CHECKPOINT",
        "train_file": data_files[benchmark]["train_file"],
        "validation_file": data_files[benchmark]["validation_file"]
    })
        
    print(benchmark)
    new_file = f"{new_dir}/eval_{benchmark}.json"
    utils.save_json(config, new_file)
    
    # break

chartqa
textvqa
mmbench
infographics_w_ocr
infographics
mmmu
mathvista
ai2d
docvqa
vqa


# New - Flywheel

In [64]:
ALIGNER_RSYNC = "/fsx_0/user/tranx/rsync"
EVAL_SBATCH = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbash.sh"
EVAL_CONFIG_OVERWRITE = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite"

OUTPUT_SFT8 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel"
OUTPUT_SFT9 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel"
OUTPUT_SFT10 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5"

In [16]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    # benchmarks=["mmmu"],
    eval_plan="eval_fix",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_fix/eval_jobs_checkpoint-100.json


In [66]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT9,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    # benchmarks=["mmmu"],
    eval_plan="eval_fix",
    checkpoint_interval=100
)

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_fix/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_fix/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flyw

In [67]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    # benchmarks=["mmmu"],
    eval_plan="eval_fix",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_fix/eval_jobs_checkpoint-100.json


In [63]:
eval_helper.get_eval_scores_all(OUTPUT_SFT8, eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,NaN,NaN,NaN,0.543,0.7979,0.5912,NaN,0.71948,0.7223,0.6822,0.771


# Flywheel

In [4]:
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'
ALIGNER_ADEL_PROD_FSDP = '/fsx_0/user/tranx/adel_prod_fsdp'
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
EVAL_CONFIG_DIR_EMB65 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65"
EVAL_CONFIG_DIR_EMB65_v5 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_v5"

OUTPUT_SFT8 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel"
OUTPUT_SFT9 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel"
OUTPUT_SFT10 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5"

# BENCHMARKS = ["mathvista", "vqa"]
BENCHMARKS = ["mmmu"]

In [7]:
# train v4 - eval v4
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v4",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_v4/eval_jobs_checkpoint-100.json


In [37]:
eval_helper.get_eval_scores_all(OUTPUT_SFT8, eval_plan="eval_v4")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [19]:
# train v4 - eval v5
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65_v5,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v5_2",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_v5_2/eval_jobs_checkpoint-100.json


In [36]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [8]:
# train v5 - eval v4
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v4",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_v4/eval_jobs_checkpoint-100.json


In [24]:
eval_helper.get_eval_jobs_record("/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a",
                                 100)

'/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/eval/eval_jobs_checkpoint-100.json'

In [27]:
utils.read_json('/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/evals/eval_jobs_checkpoint-200.json')

OrderedDict([('mmmu', OrderedDict([('200', 21877)])),
             ('docvqa', OrderedDict([('200', 21878)])),
             ('mathvista', OrderedDict([('200', 21879)])),
             ('ai2d', OrderedDict([('200', 21880)])),
             ('chartqa', OrderedDict([('200', 21881)])),
             ('vqa', OrderedDict([('200', 21882)])),
             ('textvqa', OrderedDict([('200', 21883)])),
             ('infographics_w_ocr', OrderedDict([('200', 21884)])),
             ('infographics', OrderedDict([('200', 21885)])),
             ('mmbench', OrderedDict([('200', 21886)]))])

In [29]:
utils.read_json("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28/evals/eval_jobs_checkpoint-200.json")

OrderedDict([('mmmu', OrderedDict([('200', 14227)])),
             ('docvqa', OrderedDict([('200', 14228)])),
             ('mathvista', OrderedDict([('200', 14229)])),
             ('ai2d', OrderedDict([('200', 14230)])),
             ('chartqa', OrderedDict([('200', 14231)])),
             ('vqa', OrderedDict([('200', 14232)])),
             ('textvqa', OrderedDict([('200', 14233)])),
             ('infographics_w_ocr', OrderedDict([('200', 14234)])),
             ('infographics', OrderedDict([('200', 14235)])),
             ('mmbench', OrderedDict([('200', 14236)]))])

In [35]:
eval_helper.get_eval_scores_all(OUTPUT_SFT10, eval_plan="eval_v4")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.5656,0.5556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
EVAL_CONFIG_DIR_EMB65_v5

'/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_v5'

In [30]:
# train v5 - eval v5
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65_v5,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v5_bz32_ep1_train_bz32",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_v5_bz32_ep1_train_bz32/eval_jobs_checkpoint-100.json


In [34]:
eval_helper.get_eval_scores_all(OUTPUT_SFT10, eval_plan="eval_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


# r26a_flywheel

In [5]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT9,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v4"
)

New checkpoints: [100, 200, 300, 400]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-400.json


In [33]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_v4")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [6]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT9,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65_v5,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v5"
)

New checkpoints: [100, 200, 300, 400]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-400.json


In [32]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
